In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import numpy as np
import pandas as pd
from prophet import Prophet
from forecastTable import forecastTable
from columnCleaner import columnCleaner
import plotly.graph_objects as go
import plotly.express as px

In [39]:
train_obs = pd.read_csv("data/observations_train.csv")
test_obs = pd.read_csv("data/observations_test.csv")
series = pd.read_csv("data/series.csv")

normalize = lambda col: (col-col.mean())/col.std()
normed_obs = train_obs.pivot_table(values="value", index="date", columns="series_id")
normed_obs = normed_obs.apply(normalize, axis=0)

In [40]:
cols = list(set(series['series_id'].unique()) - set(['SP500']))
# cols = ['GVIPT54000S']
# cols = ["PCUADLVWRADLVWR",
# "PCUAINFOAINFO",
# "PCUAMUMAMUM",
# "PCUARETTRARETTR",
# "PCUASHCASHC",
# "PCUASTDSVASTDSV",
# "PCUATRADEATRADE",
# "PCUATRANSATRANS",
# "PCUATRNWRATRNWR",
# "PCUATTDSVATTDSV",
# "PCUAWHLTRAWHLTR",
# "PCUOMFGOMFG",
# "PCUOMINOMIN"]
features = columnCleaner("data/observations_train.csv", columns=cols)

Dropping Column: SOFR
Dropping Column: SOFRVOL
Dropping Column: DEXJPUS


In [41]:
m = Prophet()
dropped_cols = ["DEXJPUS", "SOFR", "SOFRVOL"]
for c in cols:
    if c not in dropped_cols:
        m.add_regressor(c, standardize=False)

In [42]:
m.fit(features)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [30]:
horizon = 900
cols = list(set(cols) - set(dropped_cols))
forecast = forecastTable(m,
                         horizon, 
                         "data/observations_train.csv", 
                         "data/observations_test.csv",
                        columns=cols)

Exception: Model has not been fit.

In [11]:
from prophet.utilities import regressor_coefficients

factors = regressor_coefficients(m)
factors['abs_coef'] = abs(factors['coef'])
factors.sort_values("coef", ascending=True, inplace=True)
factors

KeyError: 'beta'

In [12]:
df = px.data.tips()
fig1 = px.bar(factors, x="coef", y="regressor", orientation='h', width = 1000, height = 1200, title = "Series and Their Regression Coefficients in the Holistic Model")
fig1.show()

NameError: name 'factors' is not defined

In [13]:
t = pd.concat([normed_obs.corr()['SP500'], normalize(factors.set_index("regressor")['coef'])], axis=1)
t

NameError: name 'factors' is not defined

In [14]:
#plt.scatter(t.SP500, t.coef)
#do this in plotly
fig2 = px.scatter(t.reset_index(), x="SP500", y="coef", labels = {"SP500": "Feature Correlation with S&P 500", "coef": "Regressor Coefficient (Normalized)"}, title = "Series Correlation with S&P500 and Regressor Coefficient (Normalized)")
fig2.show()


NameError: name 't' is not defined

In [15]:
oneseventwozero = [(2.5380135361595095,
  3.4752944824347294,
  2.595920158363373,
  3.689808284365746),
 (2.5208146411166226, 3.741774828971389, 2.599282416240165, 3.571801069761113),
 (2.5632666101838217,
  3.629799022972571,
  2.6094263354521012,
  3.6193939626579774),
 (1.8564845410618511,
  2.6339568101526694,
  1.863429723686856,
  2.6667454134128956)]

In [16]:
firstten = [(3.31440167665234, 2.2019827662180855, 3.242194235334837, 2.1508165939278125),
 (3.2452444043137394,
  2.194939328679657,
  3.3123241408814135,
  2.2030626808566076),
 (3.094207117775177, 2.2192052954753296, 3.11053678319414, 2.140583631296782),
 (3.257997030812041,
  2.3917799799751442,
  3.2103402963776486,
  2.364992405569559),
 (2.4446692266326395,
  2.3078974596977107,
  2.484030211953862,
  2.370165771402371),
 (2.493468953857067,
  3.0466374812620867,
  2.511112790480943,
  3.1652605830413774),
 (2.4748201272947625,
  2.9516129580079737,
  2.491768520977864,
  2.9761009301328296),
 (2.0579406637072735,
  3.0458878177532815,
  1.8946516519733254,
  3.037771775587174),
 (2.0671830376767137,
  3.0294932596011006,
  2.1623448586130216,
  3.021137959581392),
 (2.2715921514641564,
  3.3882643641717825,
  2.2708659373803104,
  3.4582717788654382)]

In [17]:
middle = [(2.6741761903631236,
  3.6644906740139462,
  2.675022082839199,
  3.6429404403428687),
 (2.7745774602735493,
  3.9004234700232114,
  2.7806888881103125,
  3.901621858183375),
 (2.216167031886056, 3.2947802671123, 2.1479983847421718, 3.2968066622496055),
 (2.0825869949053453,
  3.1780778789374238,
  2.087507568337204,
  3.2040968228505635),
 (2.098498505246806, 3.230830313166221, 2.066464497287097, 3.0838851343611906),
 (2.55526463125446, 3.0493994847462664, 2.593707383999514, 3.434802113363648)]

In [ ]:
performance = firstten + middle + oneseventwozero
performance = pd.DataFrame(performance, columns=list(range(1, 5)))
performance.plot()

In [ ]:
performance.to_csv("model_perf.csv")

In [ ]:
wild_cards = ['UNRATE', 'INFECTDISEMVTRACKD']
top_cols = factors.sort_values("abs_coef", ascending=False)['regressor'].values
top_cols

In [ ]:
final_model = Prophet()
f = [*top_cols[:7], 'INFECTDISEMVTRACKD']
feat, mu, s = columnCleaner("data/observations_train.csv", columns=f)
train = feat.iloc[:-200]
test = feat.iloc[-200:]
dropped_cols = ["DEXJPUS", "SOFR", "SOFRVOL"]
for c in f:
    if c not in dropped_cols:
        final_model.add_regressor(c, standardize=False)

final_model.fit(train)
# cv = cross_validation(final_model, initial="30 days", period="10 days", horizon="1 day", parallel="processes")

In [ ]:
pred = final_model.predict(test.drop(columns="y"))
pred.set_index("ds", inplace=True)
pred.head()

In [116]:
fig, ax = plt.subplots(figsize=(16, 9))

# real S&P500
sp500 = train_obs.query("""series_id == 'SP500'""")
sp500.set_index(pd.to_datetime(sp500['date']), inplace=True)
# sp500['value'].plot(ax=ax, label="S&P 500")

#training data
sp500.iloc[:-141]['value'].plot(ax=ax, label="Training")

#prediction
((pred['yhat']*s) + mu).plot(ax=ax, label="Predicted")

ax.legend(loc='upper left')

NameError: name 'plt' is not defined

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=random_x, y=random_y1,
                    mode='lines+markers',
                    name='lines+markers'))
fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    mode='markers', name='markers'))

fig.show()